# Assignment-08:
Exploring variance redunction techniques, namely:
* Antithetic Variables
* Control Variates

In [1]:
# Importing Dependencies
import numpy as np
import pandas as pd
import math
from scipy.stats import norm

In [2]:
# Defining a Linear Congruence Generator
def uniform_generator(x):
    a = 1597
    c = 2330197
    m = 2**32
    x = (a*x + c)%m
    return x, x/m

## Q1:
Using a Monte Carlo estimator to estimate I = E[exp(U)]. Comparing variance redunction techniques.

### Part (a): 
In this part we have to generate I using routine Monte Carlo Methods.

In [3]:
M = [10, 100, 1000, 10000, 100000] # Given values of 'M'.
I_a = []
Var_a = []
# Part (a), Routine Generation of I_M.
for m in M:
    x = 42 # Seed set as 42.
    Y = []
    for _ in range(m):
        x, u = uniform_generator(x)
        Y.append(math.exp(u))
    mean = np.mean(Y)
    var = np.var(Y)
    I_a.append(mean)
    Var_a.append(var)

dict_a = {'M': M, 'I (routine)': I_a, 'Variance (routine)': Var_a}
df_a = pd.DataFrame(dict_a)
df_a

,M,I (routine),Variance (routine)
0,10,1.632672,0.228534
1,100,1.665897,0.206732
2,1000,1.702425,0.240016
3,10000,1.721819,0.242052
4,100000,1.720430,0.242334


### Part (b)
In this part, we have to generate I using the variance reduction technique of 'Antithetic Variables'

In [8]:
I_b = []
Var_b = []
for m in M:
    x1 = 42
    x2 = 12
    Y = []    
    for _ in range(m//2):
        x1, u1 = uniform_generator(x1)
        # x2, u2 = uniform_generator(x2)

        Y.append((math.exp(u1) + math.exp(1-u1))/2)

    mean_antithetic = np.mean(Y)
    var_antithetic = np.var(Y)

    I_b.append(mean_antithetic)
    Var_b.append(var_antithetic)

dict_b = {'M' : M, 'I (routine)': I_a, 'I (antithetic)': I_b, 'Variance (routine)': Var_a, 'Variance (antithetic)': Var_b}
df_b = pd.DataFrame(dict_b)
df_b

,M,I (routine),I (antithetic),Variance (routine),Variance (antithetic)
0,10,1.632672,1.722900,0.228534,0.006732
1,100,1.665897,1.709640,0.206732,0.004214
2,1000,1.702425,1.716349,0.240016,0.004042
3,10000,1.721819,1.718766,0.242052,0.003989
4,100000,1.720430,1.718318,0.242334,0.003923


### Part (c)
In this part we have to generate I using the variance reduction technique of 'Control Variates'.

We generate Yi's using Yi(b) = Yi − b(Xi − E[X]), where b is taken to be the optimal value such that 'b*' = Cov(X, Y)/Var(X)
Therefore, we get b* = (E[XY] - E[X]E[Y])/Var[X], 
Here X is the Uniform Distribution as it is mentioned that U is the control variable.
Therefore, b* = 12*(E[UY] - 1/2*E[Y]) which comes out to be 

= 12*(1.5 - e/2).

Thus, b\* ~ 1.6903

In [9]:
I_c = []
Var_c = []

b = 12*(1.5 - (math.e/2))
print(b)
for m in M:
    Y = []
    x = 42
    for _ in range(m):
        x, u = uniform_generator(x)
        
        y = math.exp(u)
        y = y - b*(u - 0.5) # Control variate step.
        Y.append(y)
        
    mean_cv= np.mean(Y)
    var_cv = np.var(Y)
    I_c.append(mean_cv)
    Var_c.append(var_cv)
    
dict_c = {'M' : M, 'I (routine)': I_a, 'I (antithetic)': I_b, 'I (control variates)':I_c, 'Variance (routine)': Var_a, 'Variance (antithetic)': Var_b, 'Variance (control variates)': Var_c}
df_c = pd.DataFrame(dict_c)
df_c 

1.6903090292457295


,M,I (routine),I (antithetic),I (control variates),Variance (routine),Variance (antithetic),Variance (control variates)
0,10,1.632672,1.722900,1.720554,0.228534,0.006732,0.004377
1,100,1.665897,1.709640,1.710342,0.206732,0.004214,0.003753
2,1000,1.702425,1.716349,1.718205,0.240016,0.004042,0.004043
3,10000,1.721819,1.718766,1.718220,0.242052,0.003989,0.003983
4,100000,1.720430,1.718318,1.718329,0.242334,0.003923,0.003945


### Part (d)
Comparing results and final tabulation.

The following table shows the results of the above parts along with the percentage improvement of the 2 other methods.



In [12]:
Var_red_anti = 100*(np.array(Var_a) - np.array(Var_b))/np.array(Var_a)
Var_red_cv = 100*(np.array(Var_a) - np.array(Var_c))/np.array(Var_a)
dict = {'M' : M, 'I (routine)': I_a, 'I (antithetic)': I_b, 'I (control variates)':I_c, 'Variance (routine)': Var_a, 'Variance (antithetic)': Var_b, 'Reduction Antithetic (in %)': Var_red_anti, 'Variance (control variates)': Var_c, 'Reduction CV (in %)': Var_red_cv}
df = pd.DataFrame(dict)
df 

,M,I (routine),I (antithetic),I (control variates),Variance (routine),Variance (antithetic),Reduction Antithetic (in %),Variance (control variates),Reduction CV (in %)
0,10,1.632672,1.633257,1.720554,0.228534,0.066737,70.797709,0.004377,98.084774
1,100,1.665897,1.685969,1.710342,0.206732,0.091839,55.575697,0.003753,98.184814
2,1000,1.702425,1.710554,1.718205,0.240016,0.122525,48.951173,0.004043,98.315340
3,10000,1.721819,1.721093,1.718220,0.242052,0.123266,49.074608,0.003983,98.354310
4,100000,1.720430,1.720567,1.718329,0.242334,0.121499,49.862752,0.003945,98.371958


## Q2


In [14]:
# defining a normal generator using the Box-Muller method to sample from N(0, 1)
def normal_generator(x_1, x_2):
    x_1, u1 = uniform_generator(x_1)
    x_2, u2 = uniform_generator(x_2)
    r = math.sqrt(-2*math.log(u1))
    theta = 2*math.pi*u2
    return r*math.cos(theta), r*math.sin(theta), x_1, x_2

In [15]:
M = [10, 100, 1000, 10000, 100000, 1000000] # Given values of 'M'.
# Naive Estimate
for m in M:
    x0 = 42
    x1 = 12
    x2 = 37
    count = 0
    P = []
    for _ in range(m):
        x0, u0 = uniform_generator(x0)
        y = -math.log(u0)
        z, a1, x1, x2 = normal_generator(x1, x2)
        # Now generating N(y, 4)
        x = 2*z + y
        if(x > 1):
            P.append(x)
            count += 1
    print(f"Variance: {np.var(P)}")
    print(f"Estimate of probability P(X > 1) for M = {m} is ", count/m)
    print("\n------------------------------\n")

Variance: 30.426231085174017
Estimate of probability P(X > 1) for M = 10 is  0.5

------------------------------

Variance: 6.909539201611647
Estimate of probability P(X > 1) for M = 100 is  0.5

------------------------------

Variance: 2.731210358903938
Estimate of probability P(X > 1) for M = 1000 is  0.535

------------------------------

Variance: 1.9820052717274028
Estimate of probability P(X > 1) for M = 10000 is  0.4967

------------------------------

Variance: 2.045928122781121
Estimate of probability P(X > 1) for M = 100000 is  0.49401

------------------------------

Variance: 2.051752497677875
Estimate of probability P(X > 1) for M = 1000000 is  0.491722

------------------------------



In [16]:
M = [10, 100, 1000, 10000, 100000] # Given values of 'M'.
for m in M:
    x = 42
    P = []
    for _ in range(m):
        x, u = uniform_generator(x)
        y = -math.log(u) # Generating Y using inverse transform sampling.
        p = 1 - norm.cdf((1 - y)/2)
        P.append(p)
    print("Variance ", np.var(P))
    print(f"Estimate of probability P(X > 1) for M = {m} is ", np.mean(P))
    print("\n------------------------------\n")

Variance  0.04037296379522183
Estimate of probability P(X > 1) for M = 10 is  0.5296133428123422

------------------------------

Variance  0.026074928326839282
Estimate of probability P(X > 1) for M = 100 is  0.49864741464253215

------------------------------

Variance  0.027283175574191434
Estimate of probability P(X > 1) for M = 1000 is  0.4950798025605152

------------------------------

Variance  0.026821273240473495
Estimate of probability P(X > 1) for M = 10000 is  0.48927825703668587

------------------------------

Variance  0.026763630442263455
Estimate of probability P(X > 1) for M = 100000 is  0.4896537488592553

------------------------------



In [ ]:
M = [10, 100, 1000, 10000, 100000] # Given values of 'M'.
# Antithetic generation of Y.
for m in M:
    x1 = 42
    x2 = 12
    x3 = 37
    x4 = 75
    P = []
    for _ in range(m):
        x1, u1 = uniform_generator(x1)
        x3, u3 = uniform_generator(x3)
        y1 = (-math.log(u1)) # Generating Y using inverse transform sampling. and antithetic variables.
        y2 = (-math.log(u3))
        p = ((1 - norm.cdf((1 - y1)/2)) + (1 - norm.cdf((1 - y2)/2)))/2 # Generation using antithetic variables and conditional expectation.
        P.append(p)
    print("Variance ", np.var(P))
    print(f"Estimate of probability P(X > 1) for M = {m} is ", np.mean(P))
    print("\n------------------------------\n")

Variance  0.0346477792888233
Estimate of probability P(X > 1) for M = 10 is  0.4979842597517825

------------------------------

Variance  0.01549704644304718
Estimate of probability P(X > 1) for M = 100 is  0.49840308936064004

------------------------------

Variance  0.013970731082973111
Estimate of probability P(X > 1) for M = 1000 is  0.49546252222665205

------------------------------

Variance  0.013325971753835798
Estimate of probability P(X > 1) for M = 10000 is  0.4891099533711472

------------------------------

Variance  0.01330739816940121
Estimate of probability P(X > 1) for M = 100000 is  0.48967478236789613

------------------------------



## Q3
To find the probabilty of total rainfall for a month being less than 5cm. Compare results using MC methods and stratified sampling.
First, we generate N according to the given probabilities as the number of rainfall events occuring in a month. Then, we sample N times from the 'Weib' distribution in order to get the total rainfall for that month. 
Then, we calculate probablity as per the count and total samples.
Stratified probability is calculated by calculating probability of each class and then multiplying it by the occurence probability of each class.

In [19]:
def weib_generator(x): # Sampling from the weib distribution using inverse transform sampling.
    x, u = uniform_generator(x)
    k = 0.8
    s = 3
    return x, s * (-math.log(1 - u))**(1/k)

In [20]:
P = {1: 0.19, 2: 0.26, 3: 0.24, 4: 0.17, 5: 0.14}
samples = [100, 10000, 1000000]
for n in samples:
    x1 = 42
    x2 = 12
    try1 = 0
    # try2 = 0
    store = [[] for _ in range(5)]
    prob_ss = []
    prob_mc = []
    for _ in range(n):
        # Generating N as per the given probability.
        x1, u = uniform_generator(x1)
        if(u < 0.19):
            N = 1
        elif(u < 0.45):
            N = 2
        elif(u < 0.69):
            N = 3
        elif(u < 0.86):
            N = 4
        else:
            N = 5
        
        T = 0
        for t in range(N):
            x2, w = weib_generator(x2)
            T += w
        store[N-1].append(T)
        if(T < 5):
            try1 += 1
    prob = 0
    for i in range(5):
        count = 0
        tot = 0
        for v in store[i]:
            tot += 1
            if(v < 5):
                count += 1
        prob += (P[i+1]*(count/tot))
        
    prob_ss.append(prob)
    prob_mc.append(try1 / n)
    
    std_ss = np.std(prob_ss)
    std_mc = np.std(prob_mc)
    
    # 99% Confidence intervals for both methods
    ss = (prob - 2.576 * (std_ss / np.sqrt(n)), 
                     prob + 2.576 * (std_ss / np.sqrt(n)))
    
    mc = ((try1 / n) - 2.576 * (std_mc / np.sqrt(n)), 
             (try1 / n) + 2.576 * (std_mc / np.sqrt(n)))
    

    print(f"Estimate of probability for n: {n} is {prob} using stratified methods and {try1/n} using MC methods")
    print(f"Stratified estimate: {prob} with 99% CI: {ss}")
    print(f"MC estimate: {(try1 / n)} with 99% CI: {mc}")
    print("\n------------------------------\n")

Estimate of probability for n: 100 is 0.3423333333333334 using stratified methods and 0.36 using MC methods
Stratified estimate: 0.3423333333333334 with 99% CI: (0.3423333333333334, 0.3423333333333334)
MC estimate: 0.36 with 99% CI: (0.36, 0.36)

------------------------------

Estimate of probability for n: 10000 is 0.3696082240576159 using stratified methods and 0.3679 using MC methods
Stratified estimate: 0.3696082240576159 with 99% CI: (0.3696082240576159, 0.3696082240576159)
MC estimate: 0.3679 with 99% CI: (0.3679, 0.3679)

------------------------------

Estimate of probability for n: 1000000 is 0.36543637164966525 using stratified methods and 0.365327 using MC methods
Stratified estimate: 0.36543637164966525 with 99% CI: (0.36543637164966525, 0.36543637164966525)
MC estimate: 0.365327 with 99% CI: (0.365327, 0.365327)

------------------------------



# end